In [3]:
import numpy as np
import pickle
from database import *
from croc_lib import *
import time
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Extract Contact Sequence from File

In [6]:
croc_timestep = 0.0375
zloco_timestep = 0.005
sub_fac = croc_timestep/loco_timestep #difference in time scale between locobox and crocoddyl
time_offset = 100
ROBOT = loadTalos()

In [5]:
from os import listdir
root_file = '../samples/talos_circle/'
file_names = listdir(root_file)
print len(file_names)

OSError: [Errno 2] No such file or directory: '../samples/talos_circle/'

In [4]:
def get_info(f):
    lines = f.readlines()
    alpha_raw = lines[0]
    alpha = float(alpha_raw.split(' ')[1])
    goal_raw= lines[1]

    goal = clean_data(goal_raw)
    goal = np.array([goal[0], goal[1], alpha])[np.newaxis,:]
    try:
        contact_success = lines[5].find('True') > 0
        PG_success = lines[11].find('True') > 0
    except:
        print 'Error in info for file' + f.name
        contact_success = False
        PG_success = False
    return goal, contact_success, PG_success

def clean_data(data):
    data= data.replace('[','')
    data=data.replace(']','')
    data=data.replace('=','')
    data=data.replace(',','')
    data=data.replace('\n','')
    data=data.replace('  ',' ')
    data=data.replace('  ',' ')
    return [float(d) for d in data.split(' ')[1:]]

### Retrieve the dataset used for NN 

In [5]:
from sklearn.externals import joblib
file_name = 'data/compressed_data.sav'
is_save = False

if is_save:
    #Save data
    data = dict()
    data['trajs'] = trajs
    data['goals'] = goals
    data['PG'] = PG_successes
    data['ctc'] = contact_successes
    joblib.dump(data,file_name)
    f.close()
else:
    #load data
    f = open(file_name, 'rb')
    data = joblib.load(file_name)
    trajs = data['trajs']
    goals = data['goals']
    contact_successes = data['ctc']
    PG_successes = data['PG']
    
#trajs = np.concatenate(trajs)
#goals = np.concatenate(goals)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(goals[:,2], )

### Define the RBF 

In [9]:
import matplotlib.pyplot as plt
import numpy.matlib
import scipy.stats

dof= 39
nbStates = 150
timestep = 4000
tList = np.arange(timestep)

Mu = np.linspace(tList[0]-400, tList[-1]+400, nbStates)
Sigma  = np.reshape(np.matlib.repmat(35, 1, nbStates),[1, 1, nbStates])
Sigma.shape
Phi = np.zeros((timestep, nbStates))
for i in range(nbStates):
    Phi[:,i] = 250*scipy.stats.norm(Mu[i], Sigma[0,0,i]).pdf(tList)
#plt.plot(Phi)

### Perform PCA 

In [10]:
from sklearn.decomposition import PCA
joint_pca = PCA(n_components = 200)
pose_pca = PCA(n_components = 70)

joint_pca.fit(traj_joints)
pose_pca.fit(traj_poses)

print(joint_pca.explained_variance_)
print(joint_pca.singular_values_)

print(pose_pca.explained_variance_)
print(pose_pca.singular_values_)

joints_reduced = joint_pca.transform(traj_joints)
poses_reduced = pose_pca.transform(traj_poses)

NameError: name 'traj_joints' is not defined

### Nearest Neighbour 

In [11]:
def nearest(x_i, x):
    dists = []
    for x_j in x:
        dists.append(np.linalg.norm(x_i-x_j))
    max_index = np.argmin(dists)
    return max_index

def NN_predict(x_cur, x, y):
    y_index = nearest(x_cur, x)
    y_cur = y[y_index:y_index+1,:].copy()
    return y_cur

In [ ]:
predicted_poses =  NN_predict(goal, goals, poses_reduced)
predicted_joints = NN_predict(goal, goals, joints_reduced)
predicted_poses = pose_pca.inverse_transform(predicted_poses)
predicted_poses = predicted_poses.reshape(nbStates,7)
predicted_poses = np.dot(Phi,predicted_poses)
traj_poses = predicted_poses

predicted_joints = joint_pca.inverse_transform(predicted_joints)
predicted_joints = predicted_joints.reshape(nbStates,32)
predicted_joints = np.dot(Phi,predicted_joints)
traj_joints = predicted_joints

In [12]:
file_name = 'data/sample_data_teguh/3'
warm_start_iters = []
warm_start_costs = []
cold_start_iters = []
cold_start_costs = []
tic = time.time()
for i in range(100):
    print 'Planning ' + str(i)
    index = np.random.randint(len(file_names))
    file_name = np.array(file_names)[index]
    f = open(root_file + file_name + '/infos.log','rb')
    goal, contact_success, PG_success =  get_info(f)
    f.close()
    if contact_success and PG_success:
        #Extract the information
        rfs, lfs, timesteps = extract_contact_sequence(root_file + file_name)
        trajs = extract_traj(root_file + file_name + '/talos_flatGround_config.csv', timesteps, sub_fac, time_offset)
        vel_trajs = extract_traj(root_file + file_name + '/talos_flatGround_vel.csv', timesteps, sub_fac, time_offset)
        #subsample to crocoddyl timescale
        timesteps = np.round(np.array(timesteps)/sub_fac).astype(int)
        
        #Store into crocoddyl format
        phases = store_for_crocoddyl(timesteps, lfs, rfs, trajs, vel_trajs, croc_timestep)
        pickle.dump(phases,open('data/croc_prob_files/' + file_name ,'wb'))
        
        #load from crocoddyl and run
        #warmstart using actual dataset
        problem, xs, us, ts = define_croc_problem(ROBOT,'data/croc_prob_files/' + file_name ,  num_phases = 6, is_warmstart=True)
        solver = solve_problem(ROBOT,problem,xs, us,STOP_THRESHOLD=1e-6)
        warm_start_iters.append(solver.iter)
        warm_start_costs.append(solver.cost)
        
        #warmstart using nearest neighbor
        
        '''
        phases = store_for_crocoddypl(timesteps, lfs, rfs, trajs, vel_trajs, croc_timestep)
        '''
        
        problem, xs, us, ts = define_croc_problem(ROBOT,'data/croc_prob_files/' + file_name , num_phases = 6, is_warmstart=False)
        solver = solve_problem(ROBOT,problem,xs, us,STOP_THRESHOLD=1e-6)
        cold_start_iters.append(solver.iter)
        cold_start_costs.append(solver.cost)
        
        #displayPhaseMotion(ROBOT, solver.xs, ts)
        print 'Average Warmstart iter and costs'
        print np.mean(warm_start_iters), np.mean(warm_start_costs)
        print 'Average Coldtart iter and costs'
        print np.mean(cold_start_iters), np.mean(cold_start_costs)

        
toc = time.time()
dur = toc - tic
print 
print 'Planning takes' + str(dur) + ' s'

Planning 0
Building the set of action models and warmstart from Memmo
phase 0:
  nodes: 49
  contacts:
    suppport: ['right_sole_link', 'left_sole_link']
    swing: []
phase 1:
  nodes: 32
  contacts:
    suppport: ['right_sole_link']
    swing: ['left_sole_link']
phase 2:
  nodes: 8
  contacts:
    suppport: ['right_sole_link', 'left_sole_link']
    swing: []
phase 3:
  nodes: 40
  contacts:
    suppport: ['left_sole_link']
    swing: ['right_sole_link']
phase 4:
  nodes: 8
  contacts:
    suppport: ['right_sole_link', 'left_sole_link']
    swing: []
phase 5:
  nodes: 36
  contacts:
    suppport: ['right_sole_link']
    swing: ['left_sole_link']

*** SOLVE ***
iter 	 cost 	      stop 	    grad 	  xreg 	      ureg 	 step 	 feas
   0  1.02185e+03  4.45094e-01  4.72790e+02  1.00000e-09  1.00000e-09   1.0000     1
   1  1.47624e+02  1.76393e+01  1.91762e+03  1.00000e-09  1.00000e-09   1.0000     1
   2  8.15375e+01  5.20535e+00  1.56842e+02  1.00000e-09  1.00000e-09   1.0000     1
   3  

In [13]:
print np.mean(warm_start_costs)
print np.mean(cold_start_costs)

68.4017780819
486.254082997


In [14]:
print np.mean(warm_start_iters)
print np.mean(cold_start_iters)

19.3
18.72


In [15]:
print np.std(warm_start_iters)
print np.std(cold_start_iters)

5.99749947895
6.14016286429


In [17]:
np.array(cold_start_iters) - np.array(warm_start_iters)

array([-2,  7, -1, -2, -2,  1,  2, -2,  8, -2,  0, -3, -1,  7, -4,  4,  7,
       -1,  3,  1,  1, -2, -2,  2, -1, -3,  2, -1, -6, -1, -2,  2,  1, -2,
        2,  1, 26, 43,  5, -2,  5, -3,  2,  2, -2,  4,  7,  2, -6,  7, -3,
        7,  1, -2,  2,  1, -2, -2,  9,  2,  1,  4, -3, -1,  2, -7, -7,  8,
        3, -2,  2, -1,  4, -1,  7,  4, -6,  4, -5,  2, -1, -2,  2,  0, -2,
        4,  8, -1,  6,  2, -7, -5, -2, -2, -2, -1, -2,  4])